# Chapter 10. Data Aggregation and Group Operations

In [5]:
import pandas as pd
import numpy as np

In [6]:
df=pd.DataFrame({'key1':list('aabba'),'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,2.131483,-0.370842,a,one
1,-0.521067,-1.068204,a,two
2,-1.419241,0.496776,b,one
3,-0.051054,0.143720,b,two
4,-1.398996,0.629351,a,one


In [7]:
grouped=df['data1'].groupby(df['key1'])

# このままだと、まだそれぞれの集計結果に対してどのような処理をすればよいのか与えられていない
grouped

In [8]:
grouped.mean()

key1
a    0.070473
b   -0.735148
Name: data1, dtype: float64

In [9]:
# 複数のキーで集計
means=df['data1'].groupby([df['key1'],df['key2']]).mean()
means.unstack()

key2,one,two
key1,,
a,0.366243,-0.521067
b,-1.419241,-0.051054


In [10]:
#長ささえ同じならば、あらたに与えたarrayからでも集計可能
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states,years]).mean().unstack()

,2005,2006
California,-0.521067,-1.419241
Ohio,1.040214,-1.398996


In [11]:
# 同じDataFrameオブジェクトならば、以下のように省略可能
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.366243  0.129254
     two  -0.521067 -1.068204
b    one  -1.419241  0.496776
     two  -0.051054  0.143720

In [12]:
#GroupByはiteratonを装備
for (j,k) ,group in  df.groupby(['key1','key2']):
    print(j,k)
    print(group)

a one
      data1     data2 key1 key2
0  2.131483 -0.370842    a  one
4 -1.398996  0.629351    a  one
a two
      data1     data2 key1 key2
1 -0.521067 -1.068204    a  two
b one
      data1     data2 key1 key2
2 -1.419241  0.496776    b  one
b two
      data1    data2 key1 key2
3 -0.051054  0.14372    b  two


In [13]:
# dictに変換
dic=dict((list(df.groupby('key1'))))
dic['b']

,data1,data2,key1,key2
2,-1.419241,0.496776,b,one
3,-0.051054,0.143720,b,two


In [14]:
# column方向にtypeごとに集計

dict(list(df.groupby(df.dtypes,axis=1)))

{dtype('float64'):       data1     data2
 0  2.131483 -0.370842
 1 -0.521067 -1.068204
 2 -1.419241  0.496776
 3 -0.051054  0.143720
 4 -1.398996  0.629351, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

In [15]:
 people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2,3:5]=np.nan

# columnをmappingして集計
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
# countはsizeと違って、nanを含まない
people.groupby(mapping,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [16]:
# indexを何らかの関数で変換した値でgroupBy
people.groupby(len).count()

,a,b,c,d,e
3,3,3,3,2,2
5,1,1,1,1,1
6,1,1,1,1,1


In [43]:
# groupbyしたデータに任意の関数を与える
diff=lambda x:x.max()-x.min()

df=pd.DataFrame({'key1':list('aabba'),'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})

df=df.set_index('key1')
df.groupby('key1',axis=0).agg(diff)

,data1,data2
key1,,
a,1.885399,0.544122
b,2.364502,1.756367


In [69]:
# tipsについて、日ごとと、喫煙者かで変化があるか分析
tips=pd.read_csv('./examples/tips.csv')

tips['tips_pct']=tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tips_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [74]:
grouped=tips.groupby(['day','smoker'])
# 平均を取得
grouped.agg('mean')

total_bill       tip      size  tips_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [81]:
# pctのみに焦点をあって、mean,std,最小最大の差を取得
grouped_pct=grouped['tips_pct']
max_min=lambda x:x.max()-x.min()
grouped_pct.agg(['mean','std',('min_max',max_min)])

mean       std   min_max
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [86]:
# columnごとに複数の関数を適用
funcs=['min','max','mean','count',('min_max',max_min)]
grouped['tips_pct','total_bill'].agg(funcs)

tips_pct                                     total_bill         \
                  min       max      mean count   min_max        min    max   
day  smoker                                                                   
Fri  No      0.120385  0.187735  0.151650     4  0.067349      12.46  22.75   
     Yes     0.103555  0.263480  0.174783    15  0.159925       5.75  40.17   
Sat  No      0.056797  0.291990  0.158048    45  0.235193       7.25  48.33   
     Yes     0.035638  0.325733  0.147906    42  0.290095       3.07  50.81   
Sun  No      0.059447  0.252672  0.160113    57  0.193226       8.77  48.17   
     Yes     0.065660  0.710345  0.187250    19  0.644685       7.25  45.35   
Thur No      0.072961  0.266312  0.160298    45  0.193350       7.51  41.19   
     Yes     0.090014  0.241255  0.163863    17  0.151240      10.34  43.11   

                                      
                  mean count min_max  
day  smoker                           
Fri  No      18.420000     4   10.29  
     Yes     16.813333    15   34.42  
Sat  No      19.661778    45   41.08  
     Yes     21.276667    42   47.74  
Sun  No      20.506667    57   39.40  
     Yes     24.120000    19   38.10  
Thur No      17.113111    45   33.68  
     Yes     19.190588    17   32.77

In [91]:
# columnごとに別々の関数を適用
grouped.agg({'tips_pct':'mean','size':'count'})

tips_pct  size
day  smoker                
Fri  No      0.151650     4
     Yes     0.174783    15
Sat  No      0.158048    45
     Yes     0.147906    42
Sun  No      0.160113    57
     Yes     0.187250    19
Thur No      0.160298    45
     Yes     0.163863    17

In [99]:
def top(df,n=5,column='tips_pct'):
    return df.sort_values(by=column)[-n:] 

# それぞれの集合から、上5つを取得
tips.groupby('smoker').apply(top,column='size')

total_bill   tip smoker   day    time  size  tips_pct
smoker                                                           
No     155       29.85  5.14     No   Sun  Dinner     5  0.172194
       143       27.05  5.00     No  Thur   Lunch     6  0.184843
       141       34.30  6.70     No  Thur   Lunch     6  0.195335
       156       48.17  5.00     No   Sun  Dinner     6  0.103799
       125       29.80  4.20     No  Thur   Lunch     6  0.140940
Yes    180       34.65  3.68    Yes   Sun  Dinner     4  0.106205
       219       30.14  3.09    Yes   Sat  Dinner     4  0.102522
       56        38.01  3.00    Yes   Sat  Dinner     4  0.078927
       216       28.15  3.00    Yes   Sat  Dinner     5  0.106572
       187       30.46  2.00    Yes   Sun  Dinner     5  0.065660

In [109]:
# それぞれの集合から復元無作為抽出
def sample(df,n=10):
     return df.sample(n,replace=True)
    
tips.groupby('smoker').apply(sample,n=5)

total_bill   tip smoker   day    time  size  tips_pct
smoker                                                           
No     155       29.85  5.14     No   Sun  Dinner     5  0.172194
       71        17.07  3.00     No   Sat  Dinner     3  0.175747
       135        8.51  1.25     No  Thur   Lunch     2  0.146886
       8         15.04  1.96     No   Sun  Dinner     2  0.130319
       48        28.55  2.05     No   Sun  Dinner     3  0.071804
Yes    63        18.29  3.76    Yes   Sat  Dinner     4  0.205577
       236       12.60  1.00    Yes   Sat  Dinner     2  0.079365
       101       15.38  3.00    Yes   Fri  Dinner     2  0.195059
       138       16.00  2.00    Yes  Thur   Lunch     2  0.125000
       107       25.21  4.29    Yes   Sat  Dinner     2  0.170171

In [ ]:
# TODO 10.4 Pivot Tables and Cross-Tabulation